In [10]:
import matplotlib.pyplot as plt
import pysal
import pandas as pd
import shapely
import geopandas as gpd

In [4]:
def convertGeometries(df):
    for i in list(df.index):
        geom_type = df.loc[i, 'GeometryType']
        geom = None
        if geom_type == 'Point':
            geom =shapely.geometry.Point(df.loc[i,'Coordinates'])
        elif geom_type == 'GeometryCollection':
            if df.loc[i,'Coordinates'][0] == df.loc[i,'Coordinates'][-1]:
                geom = shapely.geometry.Polygon(df.loc[i,'Coordinates'])
            else:
                geom = shapely.geometry.LineString(df.loc[i,'Coordinates'])
        df.loc[i, 'geometry'] = geom

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/robisoniv/navwarning-analysis/master/CleanedData.csv')
# df = df.sample(frac=0.05)
df = df.drop(columns=['Unnamed: 0'])
df['StartDate'] = pd.to_datetime(df['StartDate'])
df['EndDate'] = pd.to_datetime(df['EndDate'])
df['BroadcastingDate'] = pd.to_datetime(df['BroadcastingDate'])
df['Coordinates'] = df['Coordinates'].apply(lambda c: eval(c))
convertGeometries(df)

In [ ]:
df_sample = df.sample(n=100)

In [11]:
gdf = gpd.GeoDataFrame(df_sample)
gdf['centroid'] = gdf['geometry'].centroid

for i in list(gdf.index):
    gdf.loc[i, 'lon_centroid'] = list(gdf.loc[i, 'centroid'].coords[0])[0]
    gdf.loc[i, 'lat_centroid'] = list(gdf.loc[i, 'centroid'].coords[0])[1]

In [76]:
pts = gdf[['lon_centroid', 'lat_centroid']].values

In [77]:
# Generate the W object
# https://pysal.readthedocs.io/en/latest/generated/pysal.lib.weights.KNN.html#pysal.lib.weights.KNN

kd = pysal.lib.cg.kdtree.KDTree(pts)
wnn = pysal.lib.weights.KNN(kd, 10)

In [78]:
# Instantiate an array indicating the presence of each point
# This is because we don't have a variable associated with each point

y = np.ones(len(wnn.weights))

In [89]:
mi = pysal.explore.esda.moran.Moran(y, wnn)
gc = pysal.explore.esda.geary.Geary(y, wnn)

In [90]:
print("Moran's I:\t\t", mi.I)
print("Moran's I p:\t\t", mi.p_norm)
print("Geary's C:\t\t", gc.C)
print("Geary's C p:\t\t", gc.p_norm)

Moran's I:		 nan
Moran's I p:		 nan
Geary's C:		 nan
Geary's C p:		 nan


In [ ]:
# Questions
# What am I doing wrong?
# Is my y array wrong? 
# Maybe I'm using the wrong number of neighbors in the KNN?

In [85]:
# Example from https://pysal.readthedocs.io/en/latest/generated/pysal.explore.esda.Moran.html#pysal.explore.esda.Moran

import pysal.lib
import numpy as np
w_ex = pysal.lib.io.open(pysal.lib.examples.get_path("stl.gal")).read()
f_ex = pysal.lib.io.open(pysal.lib.examples.get_path("stl_hom.txt"))
y_ex = np.array(f_ex.by_col['HR8893'])
from pysal.explore.esda.moran import Moran
mi_ex = Moran(y_ex,  w_ex)


In [86]:
print("Moran's I:\t\t", mi_ex.I)
print("Moran's I p:\t\t", mi_ex.p_norm)

Moran's I:		 0.24365582621771695
Moran's I p:		 0.00027147862770937614
